Makes a little video of what happens when $a$ increases in the Pareto distribution of radii
$$
p_Y(x) = \frac{a m^a}{x^{a+1}}
$$
for $x \geq 1$, $p_Y(x) = 0$ for $x < 1$.

If $a > 2$ this has second moment
$$
\mathbb{E}[Y^2] = \frac{a m^a}{a-2}.
$$

In [1]:
import numpy as np
from scipy.spatial import KDTree
from tqdm import tqdm
import networkx
import colorspace
from PIL import Image, ImageColor
import sys

from unconstrained import sample_points
from draw_jm import get_adjacency, colour_graph, get_ball_pixels

In [2]:
def assign_cells_random_radii(seeds, rates, img_size, T=1.0):
    min_cov_times = np.full((img_size,img_size),np.inf) # running minimum coverage times
    assignments = np.empty((img_size,img_size),dtype=int)
    for i in range(len(rates)):
        xi = seeds[i]
        gi = rates[i]
        gi2 = gi*gi
        indices, d2s = get_ball_pixels(xi, T*gi, img_size)
        for k, ij_pair in enumerate(indices):
            cov_time2 = d2s[k] / gi2
            if cov_time2 < min_cov_times[ij_pair]:
                assignments[ij_pair] = i
                min_cov_times[ij_pair] = cov_time2
    return assignments

In [3]:
def pareto_frame(seeds, a, m, fileprefix):
    rates = m*(np.random.pareto(a,size=seeds.shape[0])+1)
    if a <= 2:
        second_moment = 1
    else:
        second_moment = a*m**a / (a - 2)
    max_time = 2*np.sqrt( np.log(n) / (np.pi * n * second_moment) )
    I = assign_cells_random_radii(seeds, rates, resolution, T = max_time)

    cell_structure = get_adjacency(I)
    print(cell_structure)

    colours = colour_graph(cell_structure)
    print(f'We have a {max(colours.values())+1}-colouring of the cells.')

    c = colorspace.hcl_palettes().get_palette(name="SunsetDark")
    hex_colours = c(max(colours.values())+1)
    rgb_colours = [ImageColor.getcolor(col,"RGB") for col in hex_colours]

    data = np.empty((resolution, resolution, 3), dtype=np.uint8)
    N = I.shape[0]
    for i in range(N):
        for j in range(N):
            data[i,j,:] = rgb_colours[colours[I[i,j]]]

    image = Image.fromarray(data)
    image.save(fileprefix+str(a)+'.png')

In [ ]:
fileprefix = "frames/pareto-"
n = 100
resolution = 800

RANDOMSEED = 20240416

np.random.seed(RANDOMSEED)

seeds = sample_points(n)

for a in tqdm(np.linspace(1,4,num=10)):
    print(a)
    if a>2:
        m = ((a-2)/a)**(1/a)
    else:
        m = 1.0
    np.random.seed(RANDOMSEED) # reset the seed so the radii are close to the previous radii
    pareto_frame(seeds, a,m, fileprefix)


The colours will "flicker" in the video because the adjacency graph changes when the radii change. So we create a supremum of the adjacency graphs, in which two vertices are connected if the corresponding cells are adjacent in _any_ diagram. Then we can colour according to this for each diagram.

That graph isn't planar any more, so the number of colours may grow fast.

A disadvantage of this method is there's a very long wait before any frames appear. Or maybe that's an advantage: sometimes I spend far too long checking "how are my frames coming along?"

---

One interesting consequence of the phase transition around $a=2$: it's pretty noticeable that the assignment runs much faster when $a > 2$ than for $a \leq 2$.

In [ ]:
fileprefix = "frames/pareto-"
n = 100
resolution = 800
nframes = 100
exponents = np.linspace(4,0,num=nframes,endpoint=False)
RANDOMSEED = 20240417

np.random.seed(RANDOMSEED)

seeds = sample_points(n)

assignments = []
progress = tqdm(exponents)
for a in progress:
    progress.set_description(f'Working on a={a:.4f}')
    if a>2:
        m = ((a-2)/a)**(1/a)
    else:
        m = 1.0
    np.random.seed(RANDOMSEED)
    rates = m*(np.random.pareto(a,size=seeds.shape[0])+1)
    max_time = 2*np.sqrt( np.log(n) / (np.pi * n) )    
    assignments.append(assign_cells_random_radii(seeds, rates, resolution, T = max_time))

supG = get_adjacency(assignments[0])
for i in range(1,len(assignments)):
    supG.update(get_adjacency(assignments[i]))
print(supG)
colours = colour_graph(supG)
print(f'We have a {max(colours.values())+1}-colouring of the cells.')
# It's all deterministic after this point.
c = colorspace.hcl_palettes().get_palette(name="SunsetDark")
hex_colours = c(max(colours.values())+1)
rgb_colours = [ImageColor.getcolor(col,"RGB") for col in hex_colours]
print("Drawing the frames.")
for i, a in tqdm(enumerate(exponents)):
    I = assignments[i]
    data = np.empty((resolution, resolution, 3), dtype=np.uint8)
    for x in range(resolution):
        for y in range(resolution):
            data[x,y,:] = rgb_colours[colours[I[x,y]]]
    image = Image.fromarray(data)
    image.save(fileprefix+str(a)+'.png')

Working on a = 1.3200:  67%|████████████████████████████▊              | 67/100 [08:51<08:26, 15.33s/it]